In [ ]:
streamlit
===========
open-source framework - interactive webapplications for data science and ML Projects
Interactive Widgets
Deployment
No html/js/css required

import streamlit
streamlit.function() -> UI

In [2]:
! streamlit run s1.py

^C


In [3]:
! streamlit run s2.py

^C


In [4]:
! streamlit run s3.py

^C


In [5]:
! streamlit run s4.py

^C


In [6]:
! streamlit run s5.py

^C


In [7]:
! streamlit run s6.py

^C


In [8]:
! streamlit run s7.py

^C


In [ ]:
student --enquire PG Course -------
    |____friend1 -- CS      ------  1
    |____teacher -- Civil        
    |____Family1 -- Physics  
    |____Family2 -- CS              1      CS = 3
    |____friend2 -- Mech
    |____teacher -- CS              1   

In [11]:
! streamlit run randomForestML.py

^C


In [ ]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

st.title("Linear Regression Demo")

# Sidebar inputs
st.sidebar.header("Data Settings")
n_points = st.sidebar.slider("Number of data points", 10, 200, 50)
noise_level = st.sidebar.slider("Noise level", 0.0, 10.0, 2.0)

# Generate synthetic data
np.random.seed(42)
X = np.linspace(0, 10, n_points)
y = 3 * X + 7 + np.random.randn(n_points) * noise_level

# Reshape X for sklearn
X_reshaped = X.reshape(-1, 1)

# Fit linear regression
model = LinearRegression()
model.fit(X_reshaped, y)

# Predict
y_pred = model.predict(X_reshaped)

# Show model parameters
st.write(f"**Slope (Coefficient):** {model.coef_[0]:.2f}")
st.write(f"**Intercept:** {model.intercept_:.2f}")

# Plot
fig, ax = plt.subplots()
ax.scatter(X, y, label="Data", color="blue")
ax.plot(X, y_pred, color="red", label="Regression Line")
ax.set_xlabel("X")
ax.set_ylabel("y")
ax.legend()
st.pyplot(fig)

# Show data table
df = pd.DataFrame({"X": X, "y": y, "Predicted y": y_pred})
st.dataframe(df)

In [12]:
! streamlit run linearRegression_strem.py

^C


In [13]:
! streamlit run app_search_engine.py

^C


In [ ]:
# RAG Example
#--------------------
# File: Sqlite3_Ollama.py 
import sqlite3
import ollama

# 1. Set up SQLite database and store sample documents
def setup_database():
    conn = sqlite3.connect('myfile.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS documents (
                        id INTEGER PRIMARY KEY,
                        content TEXT
                      )''')
    sample_documents = [
        ("The first letter is alpha.",),
        ("The second letter is beta.",)
    ]
    cursor.executemany("INSERT INTO documents (content) VALUES (?)", sample_documents)
    conn.commit()
    conn.close()

setup_database()

# 2. Function to retrieve documents based on query
def retrieve_documents(query):
    conn = sqlite3.connect('myfile.db')
    cursor = conn.cursor()
    cursor.execute("SELECT content FROM documents WHERE content LIKE ?", ('%' + query + '%',))
    results = cursor.fetchall()
    conn.close()
    return [result[0] for result in results]

# 3. Function to generate a response using Ollama
def generate_response(query, documents):
    context = "\n".join(documents)
    prompt = f"Query: {query}\n\nContext:\n{context}\n\nResponse:"
    response = ollama.chat(model="gemma:2b", messages=[{"role": "user", "content": prompt}])
    return response['message']

# 4. Complete flow: Query input, document retrieval, and response generation
def handle_query(query):
    retrieved_docs = retrieve_documents(query)
    if retrieved_docs:
        response = generate_response(query, retrieved_docs)
        return response
    else:
        return "No relevant documents found."

# Test the whole process with a sample query
query = "first"
response = handle_query(query)
print(f"Response: {response}")

In [ ]:
##
## Folder_Search.py
import streamlit as st
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.agents import initialize_agent, Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_groq import ChatGroq


# Load environment
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")


# Init Streamlit
st.set_page_config(page_title="Search Engine with LangChain Agents")
st.title("🔍 LangChain-Powered Search Engine (Docs + Web)")
st.write("Ask questions, and the agent will choose between local documents or web search to answer.")

# Upload documents
if "vectorstore" not in st.session_state:
    with st.spinner("Loading documents and building vector index..."):
        loader = PyPDFDirectoryLoader("docs")
        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        split_docs = splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        st.session_state.vectorstore = FAISS.from_documents(split_docs, embeddings)

# Setup tools
retriever_tool = Tool(
    name="DocumentSearch",
    func=st.session_state.vectorstore.as_retriever().get_relevant_documents,
    description="Use this to search through uploaded research papers."
)

web_search_tool = Tool(
    name="WebSearch",
    func=DuckDuckGoSearchRun().run,
    description="Use this to search the web for recent or general info."
)

tools = [retriever_tool, web_search_tool]

# Choose model (OpenAI, Groq, Ollama)
# llm = ChatOpenAI(temperature=0)  # Replace with ChatGroq(...) or ChatOllama(...) if desired

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Query input
query = st.text_input("Ask a question (e.g., 'What does the research say about Llama 3?')")

if query:
    with st.spinner("Thinking..."):
        answer = agent_executor.run(query)
        st.success("Answer:")
        st.write(answer)